# Análisis de datos de nosepoke

## 0. Importación de librerías

In [1]:
import pandas as pd
import os
import docx2txt
from datetime import datetime, timedelta
# from ydata_profiling import ProfileReport

In [2]:
path="D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/"
pathMachos=f"{path}/MACHOS/"
pathHembras=f"{path}/HEMBRAS/"

## 1. Extraer lista de los registros

Lo primero que se hará es extraer la lista de los archivos que hay disponibles. Para esto, se recorrerán de forma iterativa las carpetas donde están los datos almacenados

In [3]:
def listaDeRegistros(base):
    """
    Toma una ubicación y realiza un proceso iterativo buscando en todos los 
    archivos que se encuentren almacenados
    
    Parameters:
    base (str): dirección del directorio base donde se buscarán los archivos que
    haya presentes

    Returns:
    registros (Array of str): lista de registros presentes en la carpeta

   """
    registros=[]
    for i in os.listdir(base):
        if os.path.isdir(base+i):
            registros+=listaDeRegistros(base+i+"/")
        elif os.path.isfile(base+i):
            registros.append(base+i)
    return registros

In [6]:
def listaDeAnimales(registros, sexChar):
    """
    Toma el nombre de archivo de un registro y la divide, extrayendo los datos
    correspondientes, asumiendo que cumple con la convención de formato
    [DD].[MM].[AAAA].[Sex]_C[#Caja]_[#Animal]_[IdPrueba]_D[#dia]_B[#Bloque].[optional].docx
    
    Parameters:
    registros (Array of str): lista de registros
    sexo (Char): Caracter que puede ser 'H' para hembras y 'M' para machos

    Returns:
    data (Pandas dataframe): dataframe de pandas con los datos de cada registro
   """
   
    dia=[]
    mes=[]
    año=[]
    sexo=[]
    caja=[]
    marca=[]
    tipoPrueba=[]
    diaEnsayo=[]
    bloque=[]
    nombreArchivo=[]
    etiqueta=[]
    
    for registro in registros:
        try:
            # Ejemplo de registro '3.12.2021.H_C21_5_REV_AZ_D8_B1.docx'
            direccion=registro.split("/")[-1:][0].split(".")
            
            # '3' '12' '2021'
            dia.append(int(direccion[0]))
            mes.append(int(direccion[1]))
            año.append(int(direccion[2]))
            
            # 'H_C21_5_REV_AZ_D8_B1'
            metadata=[]
            for i in direccion[3].split('_'):
                i = i.strip()
                if ' ' in i.strip():
                    for j in i.split(' '):
                        if j!='':
                            metadata.append(j)
                else:
                    metadata.append(i)
                        
            
            if sexChar in metadata[0]:
                sexo.append(metadata.pop(0).strip())
            else:
                sexo.append(sexChar)
                # print("Error al leer el sexo del animal: "+registro)
                
            if 'C' in metadata[0]:
                caja.append(metadata.pop(0).strip()[1:])
            else:
                caja.append(-1)
                print("Error al leer el identificador de la caja: "+registro)
                
            try:
                int(metadata[0])
                marca.append(metadata.pop(0).strip())
            except:
                marca.append(-1)
                print("Error al leer el numero del animal: "+registro)
                
            if 'B' in metadata[-1]:
                bloque.append(metadata.pop().strip()[1:])
            else:
                bloque.append(-1)
                print("Error al leer el bloque: "+registro)
                
            if 'D' in metadata[-1]:
                diaEnsayo.append(metadata.pop().strip()[1:])
            else:
                diaEnsayo.append(-1)
                print("Error al leer el bloque: "+registro)
            
            tipo=""
            for i in metadata:
                tipo+=(i.strip()+'_')
            tipoPrueba.append(tipo.strip()[:-1])
            
            if len(direccion)==6:
                etiqueta.append(direccion[4].strip())
            else:
                etiqueta.append('')
            
            nombreArchivo.append(registro)
            
        except:
            print("Error adicional: "+registro)
    d={'dia':dia, 'mes':mes, 'año':año, 'sexo':sexo, 'caja':caja,
       'marca':marca, 'tipoPrueba':tipoPrueba, 'diaEnsayo':diaEnsayo,
       'bloque':bloque, "nombreArchivo":nombreArchivo, 'etiqueta':etiqueta}
    data=pd.DataFrame(data=d)
    return data

In [4]:
def estandarizarTerminosTipoDePrueba(data):
    """
    Se tienen variaciones del identificador usado para los 7 tipos de prueba
    que se realizaron. Este método unifica los diferentes términos usados
    
    Parameters:
    data (Pandas dataframe): dataframe de pandas con los datos de cada registro

    Returns:
    cleanData (Pandas dataframe): dataframe de pandas con los términos unificados para la columna tipoPrueba
    """
    cleanData=data.copy()
    replaceDict={"_HAB":"HAB",
                 "P_DER":"SM_ADQ_DER", "SM_P_DER":"SM_ADQ_DER",
                 "SM_PRE_DER":"SM_ADQ_DER", "SM_ADQ_DER_":"SM_ADQ_DER","SM_DER_ADQ":"SM_ADQ_DER",
                 "SMM_ADQ_DER":"SM_ADQ_DER",
                 "P_IZQ":"SM_ADQ_IZQ","SM_P_IZQ":"SM_ADQ_IZQ","SM_IZQ_ADQ":"SM_ADQ_IZQ",
                 "SM_IZQ":"SM_ADQ_IZQ","SM_PRE_IZQ":"SM_ADQ_IZQ",
                 
                 "R_DER":"SM_REV_DER","SM_R_DER":"SM_REV_DER",
                 "R_IZQ":"SM_REV_IZQ","SM_R_IZQ":"SM_REV_IZQ",
                 
                 "DIS":"RA_ADQ", "DIS_AZ":"RA_ADQ", "ADQ_AZ":"RA_ADQ",
                 
                 "REV_AZ":"RA_REV", "RA":"RA_REV", "REV":"RA_REV", "R_AZ":"RA_REV",
                 }
    cleanData['tipoPrueba'].replace(to_replace=replaceDict, inplace=True)
    return cleanData

In [7]:
registrosHembras=estandarizarTerminosTipoDePrueba(listaDeAnimales(listaDeRegistros(pathHembras), 'H'))

Error adicional: D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//HEMBRAS/ACTUALIZADA. REGISTRO APRENDIZAJE EXPERIMENTO -iMac de Diego.xlsx
Error adicional: D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//HEMBRAS/Cronograma Actividades Experimento.xlsx


In [20]:
registrosHembras.head()

,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,27,8,2021,H,51,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,27,8,2021,H,51,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,27,8,2021,H,51,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,27,8,2021,H,51,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,27,8,2021,H,51,3,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,


In [8]:
registrosMachos=estandarizarTerminosTipoDePrueba(listaDeAnimales(listaDeRegistros(pathMachos), 'M'))

In [9]:
registrosMachos.head()

,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,13,8,2021,M,10,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,13,8,2021,M,10,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,13,8,2021,M,10,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,13,8,2021,M,12,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,MD


In [10]:
registrosUnificados=pd.concat([registrosMachos, registrosHembras], ignore_index=True)
registrosUnificados.insert(loc=0, column='IdAnimal', value=registrosUnificados.apply(lambda fila: f"{fila.sexo}-{fila.caja}-{fila.marca}", axis=1))
registrosUnificados.to_excel("registrosMachosYHembras.xlsx")

In [11]:
registrosUnificados.head()

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,M-10-1,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,M-10-1,13,8,2021,M,10,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,M-10-2,13,8,2021,M,10,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,M-10-2,13,8,2021,M,10,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,M-12-1,13,8,2021,M,12,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,MD


### 1.1. Análisis de calidad de los registros

Revisaremos los valores de cada columna para identificar la calidad de los datos

In [12]:
for column in registrosUnificados:
    if column != 'nombreArchivo':
        print({column:registrosUnificados[column].unique()})

{'IdAnimal': array(['M-10-1', 'M-10-2', 'M-12-1', 'M-12-2', 'M-11-1', 'M-11-2',
       'M-11-3', 'M-11-4', 'M-21-1', 'M-21-2', 'M-21-3', 'M-24-3',
       'M-24-4', 'M-25-1', 'M-25-3', 'M-25-4', 'M-27-3', 'M-27-4',
       'M-31-3', 'M-31-4', 'M-22-3', 'M-22-4', 'M-23-1', 'M-23-2',
       'M-26-1', 'M-26-2', 'M-28-1', 'M-28-5', 'M-33-4', 'M-33-5',
       'M-34-1', 'M-34-6', 'M-29-5', 'M-29-6', 'M-29-7', 'M-32-1',
       'M-32-2', 'M-32-3', 'M-30-1', 'M-30-2', 'M-30-3', 'M-35-2',
       'M-35-4', 'M-17-1', 'M-19-1', 'M-19-3', 'M-20-2', 'M-20-3',
       'M-13-1', 'M-13-2', 'M-15-1', 'M-15-2', 'M-16-1', 'M-16-2',
       'M-17-4', 'M-17-2', 'H-51-1', 'H-51-2', 'H-51-3', 'H-55-1',
       'H-55-2', 'H-55-3', 'H-56-4', 'H-56-5', 'H-56-6', 'H-56-7',
       'H-18-1', 'H-18-2', 'H-18-3', 'H-10-1', 'H-10-2', 'H-10-3',
       'H-11-4', 'H-11-5', 'H-11-6', 'H-20-1', 'H-20-2', 'H-20-3',
       'H-21-4', 'H-21-5', 'H-21-6', 'H-13-1', 'H-13-2', 'H-14-3',
       'H-17-1', 'H-17-2', 'H-16-1', 'H-22-1', 'H

Los datos de la columna 'etiqueta' indican errores que se encontraron luego de una revisión manual

| Etiqueta | Descripción |
| --- | --- |
| MD | Datos perdidos, están en blanco |
| TIME/TIEMPO | No tienen el registro del tiempo |
| .resumen_final | Solo arrojaron el resumen de todo el bloque (ensayo 50) |
| SPLIT/Split/split | Archivos que tienen entre 30 y 50 ensayos, los cuales no son consecutivos |
| EXCLUDE |  Machos C35_2 Y C21_2. Uno de ellos no logró el criterio de habituación en 20 días y debimos sacarlo del estudio, mientras que el otro se murió por el camino. De ese animal solo tenemos los datos de habituación que pueden servir (C35_2) |
| error de asignación | 

In [13]:
# registrosUnificados.loc[registrosUnificados['etiqueta'] == 'SPLIT']['nombreArchivo'].values
registrosUnificados.loc[registrosUnificados['etiqueta'] == 'SPLIT'].head(5)

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
289,M-23-1,10,12,2021,M,23,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
305,M-27-4,11,12,2021,M,27,4,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
342,M-23-2,12,12,2021,M,23,2,HAB,3,1,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
355,M-27-3,13,12,2021,M,27,3,RA_ADQ,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
356,M-27-3,13,12,2021,M,27,3,HAB,4,1,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT


## 2. Extraer datos de cada tipo de prueba
Vamos a extraer los datos de cada tipo de prueba. Para esto definiremos el enabezado de cada tipo de prueba que vamos a revisar y leeremos cada archivo.

In [56]:
medidas={"HAB":[["Ensayo_", "Aciertos_",
                        "PorcentajeAciertos_","TiempoRespuestasImpulsivas_", 
                        "Omisiones_", "PorcentajeOmisiones_", "Categorias_",
                        "CorrectasDerecha_","CorrectasIzquierda_","LatenciaPromedio_",
                        "MaximoAciertosSeguidos_"],
                        # 0: Numero, 1: porcentaje, 2: tiempo con "ms", -1: número de ensayo
                        [-1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0],
                        "Ensayo,Aciertos,Porcentaje aciertos,Tiempo total en Respuestas impulsivas,Omisiones,Porcentaje omisiones,Categorias,Total correctas a la derecha,Total correctas a la izquierda,Latencia promedio,Maximo de aciertos seguidos"],
         "SM_ADQ":[["Ensayo_", "Aciertos_",
                           "PorcentajeAciertos_", "TiempoRespuestasImpulsivas_",
                           "Omisiones_", "PorcentajeOmisiones_","Categorias_",
                           "LatenciaPromedio_", "Errores_", "PorcentajeErrores_",
                           "AdquisicionReglas_", "EstablecimientoReglas_", 
                           "MantenimientoReglas_","MaximoAciertosSeguidos_"],
                          [-1, 0, 1, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0],
                          "Ensayo,Aciertos,Porcentaje aciertos,Tiempo total en Respuestas impulsivas,Omisiones,Porcentaje omisiones,Categorias,Latencia promedio,Errores,Porcentaje errores,Adquisicion de reglas,Establecimiento de reglas,Mantenimiento de reglas,Maximo de aciertos seguidos"],
         }

In [14]:
def separarLinea(linea):
    """
    Cada línea de los archivos tiene una estructura "HH:MM:SS.MS -> DATOS".
    En caso de que no haya una entrada de tiempo se pone como hora '-1'
    
    Parameters:
    linea (String): linea de texto extraída del archivo de datos

    Returns:
    hora, registro (String, String): hora y datos por separado.
    """
    if len(linea.split('->'))==2:
        hora=linea.split('->')[0]
        registro=linea.split('->')[1]
    else:
        hora=-1
        registro=linea
    return hora, registro

### 2.1. Extraer datos habituación
Cada prueba de habituación cuenta con 50 ensayos. En cada uno de ellos se tienen los siguientes datos:

| Nombre columna | Descripción | Rango |
| --- | --- | --- |
| Ensayo | Número de ensayo | 1 - 50 |
| Aciertos | Número acomulado de aciertos | 0 - 50 |
| Porcentaje aciertos | | 0% - 100% |
| Tiempo total en respuestas impulsivas | Tiempo acomulado que pasa metiendo a la nariz entre ensayos | |
| Omisiones |

,Aciertos,Porcentaje aciertos,Tiempo total en Respuestas impulsivas,Omisiones,Porcentaje omisiones,Categorias,Total correctas a la derecha,Total correctas a la izquierda,Latencia promedio,Maximo de aciertos seguidos"],